<a href="https://colab.research.google.com/github/abhiiyer/Checking_Repo/blob/master/Assignment6/Assignment6A_Latest_Jun15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
######### ROHAN CODE ############
## Accuracy on test data is: 82.83 WITH 50 EPOCHS
## Accuracy on test data is: 83.41 WITH 100 EPOCHS


######## Max Accuracy on test data of Assignment6A is: 87.23% during epoch 210/300 ###########
##################  Total params: 1,369,738


!pip install -q keras
import keras

# save the final model to file
import sys
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D,Activation,GlobalAveragePooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras import optimizers
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras import regularizers
from keras.utils import plot_model
from matplotlib import pyplot as plt
from keras.utils import np_utils
import numpy as np

 
# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = cifar10.load_data()
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY
 
# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm
 
  
def plot_model(model_details):

    # Create sub-plots
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    
    # Summarize history for accuracy
    axs[0].plot(range(1,len(model_details.history['acc'])+1),model_details.history['acc'])
    axs[0].plot(range(1,len(model_details.history['val_acc'])+1),model_details.history['val_acc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_details.history['acc'])+1),len(model_details.history['acc'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    
    # Summarize history for loss
    axs[1].plot(range(1,len(model_details.history['loss'])+1),model_details.history['loss'])
    axs[1].plot(range(1,len(model_details.history['val_loss'])+1),model_details.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_details.history['loss'])+1),len(model_details.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    
    # Show the plot
    plt.show()
    
# run the test harness for evaluating a model
def run_test_harness():
    
  
  # load dataset
    trainX, trainY, testX, testY = load_dataset()
  
  # prepare pixel data
    trainX, testX = prep_pixels(trainX, testX)

  # define model
    model = define_model()
  
    mcp_save = keras.callbacks.ModelCheckpoint('Assignment6_best_model_improved.h5', save_best_only=True, monitor='val_loss', mode='min')
  
  # create data generator
    datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=45,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images
    
    #datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
    
	# prepare iterator
    it_train = datagen.flow(trainX, trainY, batch_size=32)
    
  # fit model
  #model.fit(trainX, trainY, epochs=2, batch_size=64, verbose=0)
    steps = int(trainX.shape[0] / 32)
    model_details = model.fit_generator(it_train, steps_per_epoch=steps, epochs=300, verbose=2, validation_data=(testX, testY), callbacks=[mcp_save])
    
    # evaluate model
    _, acc = model.evaluate(testX, testY, verbose=0)
    print('> %.3f' % (acc * 100.0))
	
	# save model
	  #model.save('final_model.h5')
    
    plot_model(model_details)


In [0]:
# define cnn model
def define_model():
  weight_decay = 1e-4
  model = Sequential()
  
  model.add(Conv2D(96, kernel_size = (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
  model.add(Dropout(0.2))
  model.add(Conv2D(96, kernel_size= (3, 3), activation='relu', padding='same'))
  model.add(Conv2D(96, kernel_size = (3, 3), activation='relu', padding='same', strides=(2,2)))
  model.add(Dropout(0.5))
	
  model.add(Conv2D(192, kernel_size = (3, 3), activation='relu', padding='same'))
  model.add(Conv2D(192, kernel_size = (3, 3), activation='relu', padding='same'))
  model.add(Conv2D(192, kernel_size = (3, 3), activation='relu', padding='same', strides=(2,2)))
  model.add(Dropout(0.5))
  
  model.add(Conv2D(192, (3, 3), activation='relu', padding = 'same'))
  model.add(Conv2D(192, (1, 1), activation='relu',padding='valid'))
  model.add(Conv2D(10, (1, 1), padding='valid'))

  model.add(GlobalAveragePooling2D())
  model.add(Activation('softmax'))
  
#   model.add(Conv2D(192, kernel_size = (3, 3), activation='relu', padding='same'))
#   model.add(Conv2D(10, kernel_size = (3, 3), activation='relu', padding='same'))
  
#   model.add(Conv2D(10, kernel_size = (5, 5), activation='relu'))
#   model.add(Conv2D(10, kernel_size = (4, 4), activation='relu'))
#   model.add(Flatten())
  
  
  
#   model.add(Conv2D(192, kernel_size = (1, 1), activation='relu', padding='valid'))
#   model.add(Conv2D(10, kernel_size = (1, 1), padding='valid'))
  #model.add(Conv2D(10, kernel_size = (4, 4))
#   model.add(GlobalAveragePooling2D())
  #model.add(MaxPooling2D((2, 2)))
  #model.add(Flatten())
  
  
  
#   model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
#   model.add(BatchNormalization())
#   model.add(Dropout(0.5))
#   model.add(Dense(10, activation='softmax'))
  model.summary()

  
	#compile model
  LEARN_RATE = 1.0e-4
  opt = Adam(lr=LEARN_RATE)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model
 


In [0]:
# entry point, run the test harness
run_test_harness()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_66 (Conv2D)           (None, 32, 32, 96)        2688      
_________________________________________________________________
dropout_22 (Dropout)         (None, 32, 32, 96)        0         
_________________________________________________________________
conv2d_67 (Conv2D)           (None, 32, 32, 96)        83040     
_________________________________________________________________
conv2d_68 (Conv2D)           (None, 16, 16, 96)        83040     
_________________________________________________________________
dropout_23 (Dropout)         (None, 16, 16, 96)        0         
_________________________________________________________________
conv2d_69 (Conv2D)           (None, 16, 16, 192)       166080    
_________________________________________________________________
conv2d_70 (Conv2D)           (None, 16, 16, 192)       331968    
__________